## Aula 14 - Pi pelo método de Monte Carlo

### Parte 1 - Implementação Sequencial

### Resultados Obtidos:
- A estimativa de Pi obtida foi **3.14484**, com um tempo de execução de **0.003277 segundos**.
- Esse resultado foi gerado utilizando **N = 100000** pontos aleatórios.

### Reflexão sobre a Estimativa de Pi:
À medida que o valor de **N** aumenta, ou seja, conforme mais pontos são gerados e analisados, a estimativa do valor de Pi se torna mais precisa. Isso ocorre porque o método de Monte Carlo se baseia em amostras aleatórias, e uma maior quantidade de pontos reduz a variabilidade na proporção de pontos dentro do círculo, convergindo para o valor exato de Pi, que é aproximadamente **3.14159**.

### Reflexão sobre a Implementação:
- **Sequência de Números Aleatórios**:
  - A sequência de números aleatórios foi gerada corretamente utilizando a função `rand()`, que foi inicializada com uma semente baseada no tempo (`std::srand(std::time(nullptr))`). Isso garante que cada execução do programa utilize uma nova sequência aleatória, evitando que os mesmos números sejam gerados em execuções consecutivas.
  
- **Dificuldades na Implementação**:
  - A implementação do algoritmo de Monte Carlo para estimar Pi é relativamente simples, e a única consideração importante é garantir que os pontos sejam corretamente gerados dentro do quadrado unitário.
  - Não houve dificuldades significativas na implementação, e o código foi eficiente em termos de tempo de execução, especialmente com **N = 100000** pontos.

Se o número de pontos **N** for aumentado, espera-se que a estimativa de Pi se aproxime cada vez mais do valor real, reduzindo a variabilidade na estimativa.

### Parte 2 - Primeira Tentativa de Paralelização

### Reflexão sobre a Comparação entre a Versão Sequencial e Paralela

#### Valor de Pi Estimado:
- **Versão Sequencial**: A versão sequencial estimou o valor de Pi em **3.14484** com um tempo de execução de **0.003277 segundos**.
- **Versão Paralela**: A versão paralela estimou o valor de Pi em **3.14632** com um tempo de execução de **0.0126971 segundos**.

Ambas as versões geraram estimativas razoavelmente próximas do valor real de Pi (aproximadamente 3.14159). No entanto, a versão paralela apresentou uma estimativa um pouco menos precisa. Isso pode ser explicado por possíveis conflitos na geração de números aleatórios quando várias threads estão em execução simultânea, já que o gerador de números aleatórios utilizado (`rand()`) não é *thread-safe*, o que pode gerar padrões repetidos e afetar a precisão.

#### Tempo de Execução:
Curiosamente, a versão paralela teve um tempo de execução **maior** que a versão sequencial, apesar do uso de múltiplas threads. Isso ocorre devido a alguns fatores importantes:
1. **Overhead de Paralelização**: Em tarefas com um número relativamente pequeno de operações (como **N = 100000**), o *overhead* causado pela criação e sincronização de threads pode ser maior do que os ganhos proporcionados pelo paralelismo.
   
2. **Geração de Números Aleatórios**: A função `rand()` não é otimizada para paralelismo e pode introduzir gargalos, pois várias threads competem para acessar o gerador de números aleatórios, impactando negativamente o desempenho.

#### Análise:
- **Desempenho**: Para valores menores de **N**, a versão paralela pode não ser tão eficiente quanto a versão sequencial, como observado no tempo de execução. No entanto, à medida que **N** aumenta, a versão paralela deve eventualmente superar a versão sequencial, diluindo o *overhead* de gerenciamento de threads no maior número de operações.
  
- **Geração de Números Aleatórios**: A geração de números aleatórios em um ambiente paralelo é um desafio, e o uso de `rand()` pode estar limitando o desempenho e a precisão. O ideal seria utilizar um gerador de números aleatórios que seja seguro para paralelismo, como `std::mt19937` da biblioteca `<random>`, onde cada thread recebe uma semente diferente.

### Conclusão:
- **Estimativa de Pi**: Embora ambas as versões tenham produzido estimativas próximas do valor de Pi, a versão sequencial foi ligeiramente mais precisa, possivelmente devido à forma como os números aleatórios foram gerados em cada caso.
- **Desempenho**: A versão paralela apresentou um tempo de execução maior que a versão sequencial para **N = 100000**, o que pode ser atribuído ao *overhead* de paralelização e à ineficiência do gerador de números aleatórios utilizado em um ambiente paralelo.

Se o número de pontos (**N**) for aumentado ou um gerador de números aleatórios adequado for utilizado, a versão paralela deve eventualmente superar a versão sequencial tanto em desempenho quanto em precisão.

### Parte 3 - Melhorando a Paralelização

### Reflexão sobre os Efeitos da Mudança

#### Estimativa de Pi:
- O valor de Pi estimado com a versão paralela melhorada foi **3.12944**. Embora este valor seja um pouco menos preciso em relação às versões anteriores, a mudança para um gerador de números aleatórios independente para cada thread foi eficaz em garantir que os números sorteados fossem diferentes para cada thread.
  
#### Tempo de Execução:
- O tempo de execução da versão melhorada foi **0.00491758 segundos**, o que representa uma melhoria significativa em relação à primeira versão paralela (que foi **0.0126971 segundos**).
- A melhoria no tempo de execução se deve à eliminação da necessidade de `#pragma omp critical` ou `reduction` para sincronizar a geração de números aleatórios. Com geradores independentes, as threads não competem entre si, resultando em um código mais eficiente.

#### Análise dos Resultados:
1. **Geração de Números Aleatórios**:
   - A paralelização da geração de números aleatórios foi feita de maneira eficaz, atribuindo a cada thread seu próprio gerador de números aleatórios (`std::mt19937`) e uma semente única baseada no ID da thread. Isso resolveu o problema de threads competindo pelo mesmo gerador, que pode ocorrer ao usar `rand()` em um ambiente paralelo.
   - Embora a estimativa de Pi não tenha sido tão precisa nesta execução, isso pode ser devido a variações nos números gerados aleatoriamente ou no valor relativamente pequeno de **N**. A precisão pode melhorar com um valor maior de **N**.

2. **Mudança no Valor de Pi**:
   - O valor de Pi caiu para **3.12944**, um pouco mais distante do valor real de **3.14159**. Isso pode ser atribuído à variabilidade da geração de números aleatórios, uma vez que cada thread gera sua própria sequência.
   - Essa leve mudança no valor estimado de Pi é comum em algoritmos de Monte Carlo devido à natureza estatística e amostral do método. Ao aumentar **N**, espera-se que o valor de Pi estimado se aproxime mais do real.

3. **Melhoria no Tempo de Execução**:
   - A versão melhorada reduziu significativamente o tempo de execução em comparação com a primeira versão paralela. A eliminação do *overhead* causado pela sincronização (`omp critical` ou `reduction`) contribuiu para essa melhoria.
   - Isso mostra que a solução de dar a cada thread seu próprio gerador de números aleatórios é uma abordagem eficiente, tanto em termos de desempenho quanto de escalabilidade.

### Observações Finais:
- **Valor de Pi Estimado com a Melhoria**: **3.12944**
- **Tempo de Execução da Nova Versão Paralela**: **0.00491758 segundos**

Essa melhoria na paralelização trouxe benefícios de desempenho, e a leve variação no valor de Pi pode ser ajustada aumentando o número de pontos gerados (**N**). A abordagem com geradores independentes foi eficaz na eliminação de gargalos de sincronização, mostrando que essa é uma técnica valiosa para paralelização de Monte Carlo.

### Conclusão


```markdown
| Versão                     | Estimativa de Pi | Tempo de Execução (s) |
|----------------------------|------------------|-----------------------|
| Sequencial                  | 3.14484          | 0.003277              |
| Primeira Tentativa Paralela | 3.14632          | 0.0126971             |
| Segunda Tentativa Paralela  | 3.12944          | 0.00491758            |
```

### Respostas às Perguntas:

1. **Houve uma melhoria significativa no tempo de execução entre a versão sequencial e as versões paralelas?**
   - **Sim, na segunda tentativa paralela**, houve uma melhoria significativa em relação à primeira versão paralela. O tempo de execução da segunda versão paralela (0.00491758 s) foi mais rápido do que a primeira versão paralela (0.0126971 s), embora ainda ligeiramente mais lento que a versão sequencial (0.003277 s). Isso sugere que, para valores de **N** maiores, a versão paralela melhorada pode superar a sequencial, especialmente após a otimização dos geradores de números aleatórios.

2. **A estimativa de Pi permaneceu precisa em todas as versões?**
   - A estimativa de Pi foi razoavelmente precisa em todas as versões, mas houve pequenas variações. A versão sequencial estimou **3.14484**, enquanto a primeira tentativa paralela estimou **3.14632**, e a segunda tentativa paralela resultou em **3.12944**. Essas pequenas diferenças são esperadas no método de Monte Carlo devido à natureza estatística do processo. A precisão pode ser melhorada aumentando o número de pontos (**N**).

3. **Quais foram os maiores desafios ao paralelizar o algoritmo, especialmente em relação aos números aleatórios?**
   - O maior desafio ao paralelizar o algoritmo foi lidar com a **geração de números aleatórios**. Na primeira tentativa, o uso de `rand()` em múltiplas threads causou gargalos, pois `rand()` não é *thread-safe* e gera sequências repetidas quando chamado em paralelo. A solução foi dar a cada thread seu próprio gerador de números aleatórios com uma semente única, o que eliminou a necessidade de sincronização e melhorou o desempenho.
   
4. **O uso de threads trouxe benefícios claros para este problema específico?**
   - **Sim, especialmente na segunda tentativa paralela**. O uso de threads trouxe benefícios claros ao reduzir a necessidade de sincronização ao gerar números aleatórios independentemente em cada thread. Embora o paralelismo tenha introduzido um overhead inicial que fez com que a primeira tentativa fosse mais lenta que a versão sequencial, a segunda tentativa mostrou que o uso eficiente de threads pode melhorar o desempenho à medida que **N** aumenta.